# Word2Vec Implementation from scratch

This project implements the [Word2Vec](https://arxiv.org/pdf/1310.4546) algorithm from first principles using PyTorch. The implementation includes all the key optimizations from the original paper: subsampling of frequent words, negative sampling, and hierarchical softmax alternatives.

The core idea behind Word2Vec is to learn dense vector representations of words that capture semantic relationships. Words appearing in similar contexts should have similar embeddings, enabling arithmetic operations on word meanings (e.g., "king" - "man" + "woman" ≈ "queen").

Note: While libraries like `gensim` will train faster and achieve higher accuracy due to extensive optimizations and C implementations, this project focuses on understanding the underlying mathematics and implementing the algorithm components step by step.

This implementation runs efficiently on CPU and does not require GPU acceleration.

__Requirements:__ if you're running locally, in the selected environment run the following command:

```pip install --upgrade nltk bokeh umap-learn```


In [1]:
import itertools
import random
import string
from collections import Counter
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import umap
from IPython.display import clear_output
from matplotlib import pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from tqdm.auto import tqdm as tqdma

2024-07-02 18:54:52.407584: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 18:54:52.407834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 18:54:52.479677: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 18:54:52.626537: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 18:54:54.058680: W tensorflow/compiler/tf2

In [ ]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

In [2]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

## Text Preprocessing and Tokenization

The first step in building Word2Vec is proper text preprocessing. The dataset contains punctuation, emojis, and other non-standard tokens, making a simple `str.split` inadequate for tokenization.

We use NLTK's `WordPunctTokenizer` for robust tokenization that handles these edge cases effectively.

In [3]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [4]:
data_tok = [
    tokenizer.tokenize(
        line.translate(str.maketrans("", "", string.punctuation)).lower()
    )
    for line in data
]
data_tok = [x for x in data_tok if len(x) >= 3]

In [5]:
data_tok[50]

['what',
 'tv',
 'shows',
 'or',
 'books',
 'help',
 'you',
 'read',
 'peoples',
 'body',
 'language']

## Vocabulary Construction and Skip-gram Preprocessing

We define the context window parameters and perform preprocessing to build the skip-gram model. The vocabulary is filtered to include only words that appear at least `min_count` times to reduce noise and computational complexity.

In [7]:
min_count = 5
window_radius = 5

In [8]:
vocabulary_with_counter = Counter(chain.from_iterable(data_tok))

word_count_dict = dict()
for word, counter in vocabulary_with_counter.items():
    if counter >= min_count:
        word_count_dict[word] = counter

vocabulary = set(word_count_dict.keys())
del vocabulary_with_counter

In [9]:
word_to_index = {word: index for index, word in enumerate(vocabulary)}
index_to_word = {index: word for word, index in word_to_index.items()}

## Context Pair Generation

The following code generates `(target_word, context_word)` pairs from the dataset based on the specified window size. This forms the foundation of the skip-gram training data.

In [10]:
context_pairs = []

for text in data_tok:
    for i, central_word in enumerate(text):
        context_indices = range(
            max(0, i - window_radius), min(i + window_radius, len(text))
        )
        for j in context_indices:
            if j == i:
                continue
            context_word = text[j]
            if central_word in vocabulary and context_word in vocabulary:
                context_pairs.append(
                    (word_to_index[central_word], word_to_index[context_word])
                )

print(f"Generated {len(context_pairs)} pairs of target and context words.")

Generated 40220313 pairs of target and context words.


## Subsampling of Frequent Words

To mitigate the imbalance between frequent and rare words, we implement a subsampling mechanism that reduces the occurrence of highly frequent words during training. This improves both training efficiency and the quality of rare word embeddings.

The probability of **excluding** a word from training at any given step is calculated as:
$$
P_\text{drop}(w_i)=1 - \sqrt{\frac{t}{f(w_i)}},
$$
where $f(w_i)$ is the normalized frequency of word $w_i$ and $t$ is the threshold parameter.

In [11]:
def subsample_frequent_words(word_count_dict, threshold=1e-5):
    """
    Calculates the subsampling probabilities for words based on their frequencies.

    This function is used to determine the probability of keeping a word in the dataset
    when subsampling frequent words. The method used is inspired by the subsampling approach
    in Word2Vec, where each word's frequency affects its probability of being kept.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.
    - threshold (float, optional): A threshold parameter used to adjust the frequency of word subsampling.
                                   Defaults to 1e-5.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of keeping each word.
    """
    all_words_count = sum([word_count_dict[word] for word in word_count_dict])
    
    prob_keep_dict = dict()
    for word in word_count_dict:
        prob_keep_dict[word] = min(1, np.sqrt(threshold * all_words_count / word_count_dict[word]))
    
    return prob_keep_dict

## Negative Sampling

For efficient training, the model must not only predict high probabilities for words that appear in context, but also predict low probabilities for words that do not. This is achieved through negative sampling, where we randomly select words that were not observed in the actual context.

The original paper proposes computing negative sampling probabilities according to the distribution $P_n(w)$:
$$
P_n(w) = \frac{U(w)^{3/4}}{Z},
$$

where $U(w)$ is the unigram frequency distribution and $Z$ is the normalization constant ensuring the probabilities sum to 1. The 3/4 power smooths the distribution, making frequent words less likely to be sampled compared to pure frequency-based sampling.

In [12]:
def get_negative_sampling_prob(word_count_dict):
    """
    Calculates the negative sampling probabilities for words based on their frequencies.

    This function adjusts the frequency of each word raised to the power of 0.75, which is
    commonly used in algorithms like Word2Vec to moderate the influence of very frequent words.
    It then normalizes these adjusted frequencies to ensure they sum to 1, forming a probability
    distribution used for negative sampling.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of selecting each word
            for negative sampling.
    """
    norm_const = sum([np.power(word_count_dict[word], 3 / 4) for word in word_count_dict])

    negative_sampling_prob_dict = {
        word: np.power(word_count_dict[word], 3 / 4) / norm_const for word in word_count_dict
    }  # THIS IS A PLACEHOLDER!
    return negative_sampling_prob_dict

For computational efficiency, we convert the probability dictionaries to arrays since all words are already indexed:

In [13]:
keep_prob_dict = subsample_frequent_words(word_count_dict)
assert keep_prob_dict.keys() == word_count_dict.keys()

In [14]:
negative_sampling_prob_dict = get_negative_sampling_prob(word_count_dict)
assert negative_sampling_prob_dict.keys() == negative_sampling_prob_dict.keys()
assert np.allclose(sum(negative_sampling_prob_dict.values()), 1)

In [15]:
keep_prob_array = np.array(
    [keep_prob_dict[index_to_word[idx]] for idx in range(len(word_to_index))]
)
negative_sampling_prob_array = np.array(
    [
        negative_sampling_prob_dict[index_to_word[idx]]
        for idx in range(len(word_to_index))
    ]
)

## Batch Generation with Negative Sampling

The following function generates training batches that include both positive context pairs and negative samples for efficient training:

In [16]:
def generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
):
    batch = []
    neg_samples = []

    while len(batch) < batch_size:
        center, context = random.choice(context_pairs)
        if random.random() < keep_prob_array[center]:
            batch.append((center, context))
            neg_sample = np.random.choice(
                range(len(negative_sampling_prob_array)),
                size=num_negatives,
                p=negative_sampling_prob_array,
            )
            neg_samples.append(neg_sample)
    batch = np.array(batch)
    neg_samples = np.vstack(neg_samples)
    return batch, neg_samples

In [17]:
"""
batch_size = 4
num_negatives = 15
batch, neg_samples = generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
                                                     )
"""

## Model Implementation

Now we implement the Word2Vec model.

With negative sampling, we maximize the following objective function:

$$
\mathcal{L} = \log \sigma({\mathbf{v}'_{w_O}}^\top \mathbf{v}_{w_I}) + \sum_{i=1}^{k} \mathbb{E}_{w_i \sim P_n(w)} \left[ \log \sigma({-\mathbf{v}'_{w_i}}^\top \mathbf{v}_{w_I}) \right],
$$

where:
- $\mathbf{v}_{w_I}$ is the input vector of the center word $w_I$,
- $\mathbf{v}'_{w_O}$ is the output vector of the context word $w_O$,
- $k$ is the number of negative samples,
- $P_n(w)$ is the negative sampling distribution defined above,
- $\sigma$ is the sigmoid function.

Implementation inspired by: https://github.com/blackredscarf/pytorch-SkipGram/blob/master/model.py

In [ ]:
class SkipGramModelWithNegSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.center_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim)
        initrange = (2.0 / (vocab_size + embedding_dim)) ** 0.5  # Xavier init
        self.center_embeddings.weight.data.uniform_(-initrange, initrange)
        self.context_embeddings.weight.data.uniform_(0, 0)

    def forward(self, center_words, pos_context_words, neg_context_words):
        # YOUR CODE HERE
        batch_size = len(center_words)
        num_negatives = np.shape(neg_context_words)[1]
        # u,v: [batch_size, emb_dim]
        v = self.center_embeddings(center_words)
        u = self.context_embeddings(pos_context_words)
        # positive_val: [batch_size]
        pos_scores = torch.sum(u * v, dim=1).squeeze()

        # u_hat: [batch_size, neg_size, emb_dim]
        u_hat = self.context_embeddings(neg_context_words)
        # [batch_size, neg_size, emb_dim] x [batch_size, emb_dim, 1] = [batch_size, neg_size, 1]
        # neg_vals: [batch_size, neg_size]
        neg_scores = torch.bmm(u_hat, v.unsqueeze(2)).squeeze(2)
        
        return pos_scores, neg_scores

In [18]:
device = torch.device("cpu")

In [19]:
vocab_size = len(word_to_index)
embedding_dim = 100
num_negatives = 15

model = SkipGramModelWithNegSampling(vocab_size, embedding_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05)
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=150)
criterion = nn.BCEWithLogitsLoss()

In [20]:
params_counter = 0
for weights in model.parameters():
    params_counter += weights.shape.numel()
assert params_counter == len(word_to_index) * embedding_dim * 2

In [21]:
def train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    device=device,
):
    pos_labels = torch.ones(batch_size).to(device)
    neg_labels = torch.zeros(batch_size, num_negatives).to(device)
    loss_history = []
    for step in tqdma(range(steps)):
        batch, neg_samples = generate_batch_with_neg_samples(
            context_pairs,
            batch_size,
            keep_prob_array,
            word_to_index,
            num_negatives,
            negative_sampling_prob_array,
        )
        center_words = torch.tensor([pair[0] for pair in batch], dtype=torch.long).to(
            device
        )
        pos_context_words = torch.tensor(
            [pair[1] for pair in batch], dtype=torch.long
        ).to(device)
        neg_context_words = torch.tensor(neg_samples, dtype=torch.long).to(device)

        optimizer.zero_grad()
        pos_scores, neg_scores = model(
            center_words, pos_context_words, neg_context_words
        )

        loss_pos = criterion(pos_scores, pos_labels)
        loss_neg = criterion(neg_scores, neg_labels)

        loss = loss_pos + loss_neg
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        lr_scheduler.step(loss_history[-1])

        if step % 50 == 0:
            print(
                f"Step {step}, Loss: {np.mean(loss_history[-100:])}, learning rate: {lr_scheduler._last_lr}"
            )

In [22]:
steps = 3000
batch_size = 512
train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
)

  0%|          | 0/3000 [00:00<?, ?it/s]

Step 0, Loss: 1.3862946033477783, learning rate: [0.05]
Step 50, Loss: 1.4400101806603225, learning rate: [0.05]
Step 100, Loss: 1.5922880280017853, learning rate: [0.05]
Step 150, Loss: 1.9171732306480407, learning rate: [0.05]
Step 200, Loss: 2.1969047498703005, learning rate: [0.025]
Step 250, Loss: 2.2643454599380495, learning rate: [0.025]
Step 300, Loss: 2.2053091955184936, learning rate: [0.025]
Step 350, Loss: 2.1768981289863585, learning rate: [0.0125]
Step 400, Loss: 2.130049102306366, learning rate: [0.0125]
Step 450, Loss: 2.065780996084213, learning rate: [0.0125]
Step 500, Loss: 2.024864729642868, learning rate: [0.00625]
Step 550, Loss: 1.9857913029193879, learning rate: [0.00625]
Step 600, Loss: 1.9597173810005188, learning rate: [0.00625]
Step 650, Loss: 1.9447150146961212, learning rate: [0.003125]
Step 700, Loss: 1.923474452495575, learning rate: [0.003125]
Step 750, Loss: 1.904655305147171, learning rate: [0.003125]
Step 800, Loss: 1.880057098865509, learning rate: 

## Extracting Word Embeddings

We extract the trained weight matrices to use as word vector representations. Following common practice, we use the context embedding matrix (decoder) as the final word embeddings, as it typically provides better representations.

In [157]:
t = torch.tensor([[1, 2]], dtype=torch.float32)
tn = F.normalize(t)
print(t, tn)

tensor([[1., 2.]]) tensor([[0.4472, 0.8944]])


In [158]:
_model_parameters = model.parameters()
embedding_matrix_center = next(
    _model_parameters
).detach()  # Assuming that first matrix was for central word
embedding_matrix_context = next(
    _model_parameters
).detach()  # Assuming that second matrix was for context word

embedding_matrix_center = F.normalize(embedding_matrix_center)
embedding_matrix_context = F.normalize(embedding_matrix_context)

In [171]:
torch.norm(embedding_matrix_context[2])

tensor(1.0000)

In [159]:
def get_word_vector(word, embedding_matrix, word_to_index=word_to_index):
    return embedding_matrix[word_to_index[word]]

## Embedding Quality Evaluation

Let's perform basic semantic similarity test:

In [161]:
similarity_1 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("laptop", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("macbook", embedding_matrix_context)[None, :],
)
assert similarity_1 > similarity_2

## Nearest Neighbors Analysis

Let's examine the nearest words based on cosine similarity. This helps evaluate whether semantically related words have similar embeddings:

In [162]:
def find_nearest(word, embedding_matrix, word_to_index=word_to_index, k=10):
    word_vector = get_word_vector(word, embedding_matrix)[None, :]
    dists = F.cosine_similarity(embedding_matrix, word_vector)
    index_sorted = torch.argsort(dists)
    top_k = index_sorted[-k:]
    return [(index_to_word[x], dists[x].item()) for x in top_k.numpy()]

In [163]:
find_nearest("python", embedding_matrix_context, k=10)

[('c', 0.6080822944641113),
 ('java', 0.6110564470291138),
 ('mru', 0.6166133880615234),
 ('cbcs', 0.620560884475708),
 ('ddwrt', 0.6249598264694214),
 ('famouspopular', 0.6281499266624451),
 ('wwwallbestlistcom', 0.6433080434799194),
 ('sharjah', 0.6445533037185669),
 ('electronics', 0.6458518505096436),
 ('python', 1.0)]

## Embedding Visualization

We can visualize how frequently occurring words are represented in the latent space using dimensionality reduction:

In [164]:
top_k = 5000
_top_words = sorted([x for x in word_count_dict.items()], key=lambda x: x[1])[
    -top_k - 100 : -100
]  # ignoring 100 most frequent words
top_words = [x[0] for x in _top_words]
del _top_words

In [165]:
word_embeddings = torch.cat(
    [embedding_matrix_context[word_to_index[x]][None, :] for x in top_words], dim=0
).numpy()

In [166]:
import bokeh.models as bm
import bokeh.plotting as pl
from bokeh.io import output_notebook

output_notebook()


def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.25,
    color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs,
):
    """draws an interactive plot for data points with auxilirary info on hover"""
    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color, **kwargs})

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height)
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

Loading BokehJS ...

In [167]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_embeddings)

In [168]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=top_words)

figure(id='p1053', ...)